In [4]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import openai

## Notebook to convert numerical based HLS annotations to tekst for non-binary evaluation with GPT

In [11]:
HLS_train = pd.read_csv('HLS_train_string.csv')

In [12]:
### Select only japan for evaluation of string
HLS_train_japan = HLS_train[HLS_train['id']=='COP19_japan']
HLS_train_japan

,Unnamed: 0.1,Unnamed: 0,id,Text,Relevance,Principle,Topic,Unit,Shape,RELEVANCE,PRINCIPLE,TOPIC,UNIT,SHAPE
0,0,0,COP19_japan,"Thank you, Mr. President .",0,0,0,0,0,Not relevant,not evaluated,not evaluated,not evaluated,not evaluated
1,1,1,COP19_japan,"On beha lf of the government of Japan , I wou...",0,0,0,0,0,Not relevant,not evaluated,not evaluated,not evaluated,not evaluated
2,2,2,COP19_japan,I would also like to expr ess my d eepest con...,0,0,0,0,0,Not relevant,not evaluated,not evaluated,not evaluated,not evaluated
3,3,3,COP19_japan,Mr. President: A fair and effective framewor...,2,3,1,2,2,Relevant,utilitarian,new UNFCCC policy,responsibility,equality
4,4,4,COP19_japan,"In this regard, Japan firmly supports the est...",1,0,0,0,0,Statement of intent,not evaluated,not evaluated,not evaluated,not evaluated
5,5,5,COP19_japan,Such a framework must be based on “nationally ...,2,2,1,2,3,Relevant,egalitarian,new UNFCCC policy,responsibility,equity
6,6,6,COP19_japan,I will devote myself toward the s uccessful o...,1,0,0,0,0,Statement of intent,not evaluated,not evaluated,not evaluated,not evaluated
7,7,7,COP19_japan,Mr. President: The Great East Japan Earthqua...,0,0,0,0,0,Not relevant,not evaluated,not evaluated,not evaluated,not evaluated
8,8,8,COP19_japan,Even under such circumstance both the public ...,0,0,0,0,0,Not relevant,not evaluated,not evaluated,not evaluated,not evaluated
9,9,9,COP19_japan,Our greenhouse gas emissions for the first co...,0,0,0,0,0,Not relevant,not evaluated,not evaluated,not evaluated,not evaluated


## Stringbased annotation with updated GPT_annotate and new codebook
First evaluate only for relevance

In [13]:
## Import the altered version of GPT_annotate
# Should ony present a single file with the outcomes
import gpt_annotate_string

In [14]:
# don't type the key in this file!
# create gpt_api.txt, put the key in that, and save
with open('gpt_api_key.txt', 'r') as f:
    key = f.read().strip()

In [15]:
HLS_relevance_string = HLS_train_japan[['Text', 'RELEVANCE']]
text_to_annotate = HLS_relevance_string

In [16]:
# Load codebook
with open('relevance_V1_string', 'r', encoding='utf-8') as file:
    codebook = file.read()

In [17]:
# Prepare the data for annotation
# Preparation is done with GPT-3.5-turbo - can be altered hardcoded
# The outcome of this analysis is not always the same, seems like something is going wrong here
text_to_annotate = gpt_annotate_string.prepare_data(text_to_annotate, codebook, key, prep_codebook=True)

ChatCompletion(id='chatcmpl-9RyHX5mjQ1wcFeVXOAyVrIWoV45gZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='RELEVANCE', role='assistant', function_call=None, tool_calls=None))], created=1716453607, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_729ea513f7', usage=CompletionUsage(completion_tokens=3, prompt_tokens=431, total_tokens=434))

Categories to annotate:
1) RELEVANCE


Data is ready to be annotated using gpt_annotate()!

Glimpse of your data:
Shape of data:  (37, 4)
   unique_id                                               text  \
0          0                         Thank you, Mr. President .   
1          1   On beha lf of the government of Japan , I wou...   
2          2   I would also like to expr ess my d eepest con...   
3          3   Mr. President:  A fair and effective framewor...   
4          4   In this regard, Japan firmly supports the est...   

             RELEVANCE               

In [19]:
# Annotate the data; should return 1 output including all annotation for each batch.
gpt_out_all =  gpt_annotate_string.gpt_annotate(text_to_annotate, codebook, key, seed=153, num_iterations = 5, model = "gpt-4o", temperature = 0.6, batch_size = 20, human_labels = True,  data_prep_warning = False, time_cost_warning = True)

You are about to annotate 37 text samples and the number of iterations is set to 5

iteration:  1 completed
iteration:  2 completed
iteration:  3 completed
iteration:  4 completed
iteration:  5 completed


Use of gpt-4o seems to provide somewhat constant outputs, at least a fingerprint that can be evaluated.

Outputfiles:
fingerprints_mainseed - fingerprints of each time the API is called. This should be the same, as seed and model parameters remain the same. This is not the case.
gpt_out_all_iterations_string.csv - output of all annotations by gpt. In theory, these outputs should be the same for all iterations. This is not always the case.

In [ ]:
gpt_out_all